In [ ]:
from typing import List, Tuple, Dict
from itertools import combinations

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, Aer
from qiskit.tools.visualization import plot_histogram
from qiskit import IBMQ

import numpy as np

IBMQ.load_account()

In [ ]:
def simulate_circuit(circuit, shots=1024):
    backend = Aer.get_backend('qasm_simulator')
    job = execute(circuit, backend=backend, shots=shots)
    result = job.result()
    counts = result.get_counts(circuit)
    return counts

## Easy non-generic solution

Consider the following graph:

<img src="images/simple_graph.png" width="400" height="400"/>

In [ ]:
def gcp_method_1() -> QuantumCircuit:
    """
    Method 1 of GCP
    Graph specific method. Analyze the graph and find the best set of gates to use.
    """

    # Create a Quantum Register with 5 qubits.
    q = QuantumRegister(5, "node")
    # Create a Classical Register with 5 bits.
    c = ClassicalRegister(5, "classical")
    # Create a Quantum Circuit acting on the q register
    circuit = QuantumCircuit(q, c)

    # First qubit in superposition
    circuit.h(0)
    # All the others in |1>
    circuit.x(q[1:])

    circuit.barrier()
    # Apply CNOTs to disallow nodes with the same color
    circuit.cx(0, 2) # 0 and 1 are connected. They can't have the same color
    circuit.cx(2, 4) # 2 and 4 are connected. They can't have the same color
    circuit.ccx(0, 4, 1) # 1 can't have the same color as 0 and 3
    circuit.cx(2, 3) # 2 and 3 are connected. They can't have the same color
    circuit.barrier()

    # Measure all qubits
    circuit.measure(q, c)

    return circuit

In [ ]:
qc_method_1 = gcp_method_1()

# Draw the circuit
qc_method_1.draw(output='mpl')

In [ ]:
# Execute the circuit on the qasm simulator
backend = Aer.get_backend('qasm_simulator')
job = execute(qc_method_1, backend, shots=1024)
result = job.result()

# Show the results
counts = result.get_counts(qc_method_1)
plot_histogram(counts)

## Generic Solution for 2-coloring

The circuit to solve this problem will need 2 input qubits and 1 ancilla qubit.

It is easier to consider the simplest graph, the one with only 2 nodes. The graph is shown below:

<img src="images/simple_graph_2_nodes.png"/>

To color the graph with only two colors, we need only to make sure that the colors of the two nodes are different. We start by placing the two input qubits in a quantum superposition using the $H$ gate. The state of the two qubits is now:

$$\frac{1}{\sqrt{2}}(|00\rangle + |01\rangle + |10\rangle + |11\rangle)$$

This state represents all the possible colour configurations for the two nodes. We must now consider the following scenarios:

1. If the colors are the same, i.e. the state $|00\rangle$ or $|11\rangle$, then we have to perform some operation that will flip one of the input qubits to the opposite color. This can be done by applying the $X$ gate to one of the input qubits.
2. If the colors are not the same, i.e. the state $|01\rangle$ or $|10\rangle$, then we can apply the $I$ gate to one of the input qubits.

The second scenario indicates that we must only apply the $X$ gate to one of the input qubits if the colors are the same. To do this we must compare the two qubits, save the result in the ancilla qubit and perform a controlled $X$ gate on one of the input qubits with the ancilla qubit as the control qubit. The circuit for this is shown below:

In [ ]:
def gcp_method_2(nodes: int, edges: List[Tuple[int, int]]) -> QuantumCircuit:
    """
    Method 2 of GCP
    General method for 2-coloring. 

    Args:
        nodes (int): _description_
        edges (List[Tuple[int, int]]): _description_

    Returns:
        QuantumCircuit: _description_
    """

    # Create a Quantum Register with nodes qubits.
    q = QuantumRegister(nodes)
    # Create len(edges) Quantum Registers for the ancilla qubits
    ancilla = QuantumRegister(len(edges))
    # Create a Classical Register with nodes bits.
    c = ClassicalRegister(nodes)
    # Create a Quantum Circuit
    circuit = QuantumCircuit(q, ancilla, c)

    # All input qubits in superposition
    circuit.h(q)

    circuit.barrier()
    for i, edge in enumerate(edges):
        # Check if the nodes have the same color and store the result in the ancilla qubit
        circuit.ccx(q[edge[0]], q[edge[1]], ancilla[i])

        # Perform a negated ccnot in order the cover the case where both nodes have color 0
        circuit.x(q[edge[0]])
        circuit.x(q[edge[1]])
        circuit.ccx(q[edge[0]], q[edge[1]], ancilla[i])
        circuit.x(q[edge[0]])
        circuit.x(q[edge[1]])

        # Perform controlled X on one of the input qubits using the ancilla qubit as control
        circuit.cx(ancilla[i], q[edge[1]]) # We choose 1 by default
        circuit.barrier()

    # Measure all qubits
    circuit.measure(q, c)

    return circuit

In [ ]:
# Case: 0 -- 2

[
    [0,0,1,0,0],
    [0,0,0,0,1],
    [1,0,0,1,1],
    [0,0,1,0,0],
    [0,1,1,0,0],
]

qc_method_2_case_1 = gcp_method_2(5, [(0, 2), (1, 4), (2, 3), (2, 4)])

# Draw the circuit
qc_method_2_case_1.draw(output='mpl')

In [ ]:
plot_histogram(simulate_circuit(qc_method_2_case_1))

## NK method for K-coloring

In [ ]:
from itertools import combinations

NkMeta = Tuple[
    List[Tuple[QuantumRegister, QuantumRegister]], 
    List[QuantumRegister], 
    List[QuantumRegister], 
    ClassicalRegister, 
    QuantumCircuit
]

def nk_init(num_nodes: int, num_colors: int, edges: List[Tuple[int, int]]) -> NkMeta:
    # Create a Quantum Register for each node with num_colors qubits and an ancilla qubit
    node_ancilla_registers: List[Tuple[QuantumRegister, QuantumRegister]] = []
    for node in range(num_nodes):
        tuple_register = (QuantumRegister(num_colors, f"node_{node}"), QuantumRegister(1, f"ancilla_{node}"))
        node_ancilla_registers.append(tuple_register)

    node_pairs = list(combinations(range(num_nodes), 2))

    # Create a Quantum Register for each pair of nodes with 1 qubit
    pairs_register: List = []
    for node1, node2 in node_pairs:
        pairs_register.append(QuantumRegister(1, f"pair_{node1}_{node2}"))

    # Create a Quantum Register for each edge of the graph with 1 qubit
    edge_register: List = []
    for node1, node2 in node_pairs:
        edge_register.append(QuantumRegister(1, f"edge_{node1}_{node2}"))

    # Create a Classical Register with num_nodes bits.
    classical_register = ClassicalRegister(num_nodes * num_colors, "classical")

    # Create the quantum circuit
    circuit = QuantumCircuit(
        *[reg for reg_tuple in node_ancilla_registers for reg in reg_tuple], 
        *pairs_register, 
        *edge_register,
        classical_register
    )

    for edge in edges:
        if edge in node_pairs:
            circuit.x(edge_register[node_pairs.index(edge)])
        elif (edge[1], edge[0]) in node_pairs:
            circuit.x(edge_register[node_pairs.index((edge[1], edge[0]))])
        else:
            raise Exception(f"Edge '{edge}' is not a valid edge.")

    return node_ancilla_registers, pairs_register, edge_register, classical_register, circuit

def generate_color_matrix(num_colors: int, circuit: QuantumCircuit, node_ancilla_registers: List[Tuple[QuantumRegister, QuantumRegister]]):
    # Start with all possible color arrangements
    for node, ancilla in node_ancilla_registers:
        circuit.h(node)

    for node_colors, ancilla in node_ancilla_registers:

        # Remove all arrangements where one node has more than one color
        for n1, n2 in combinations(range(num_colors), 2):
            color1 = node_colors[n1]
            color2 = node_colors[n2]

            circuit.ccx(color1, color2, ancilla)
            circuit.cx(ancilla, color1)
            circuit.reset(ancilla)

        # Remove arrangement where node has no color
        circuit.x(node_colors)
        circuit.mct(node_colors, ancilla)
        circuit.x(node_colors)
        circuit.cx(ancilla, node_colors[-1])
        circuit.reset(ancilla)
    circuit.barrier()

def detect_color_conflicts(num_nodes: int, num_colors: int, circuit: QuantumCircuit, node_ancilla_registers: List[Tuple[QuantumRegister, QuantumRegister]], pairs_register: List[QuantumRegister]):
    for n, (n1, n2) in enumerate(combinations(range(num_nodes), 2)):
        for i in range(num_colors):
            node1_color = node_ancilla_registers[n1][0][i]
            node2_color = node_ancilla_registers[n2][0][i]

            circuit.ccx(node1_color, node2_color, pairs_register[n])
    circuit.barrier()

def remove_color_conflicts(num_nodes: int, num_colors: int, circuit: QuantumCircuit, node_ancilla_registers: List[Tuple[QuantumRegister, QuantumRegister]], pairs_register: List[QuantumRegister], edge_register: List[QuantumRegister]):
    for n, (edge, pair) in enumerate(zip(edge_register, pairs_register)):
        for node, ancilla in node_ancilla_registers:
            for i in range(num_colors):
                node_color = node[i]

                circuit.mct([edge, pair, node_color], ancilla)
                circuit.cx(ancilla, node_color)
                circuit.reset(ancilla)
        # ancilla = node_ancilla_registers[n][1]
        # for i in range(num_colors):
        #     node = node_ancilla_registers[n][0][i]

        #     circuit.mct([edge, pair, node], ancilla)
        #     circuit.cx(ancilla, node)
        #     circuit.reset(ancilla)
    circuit.barrier()

def gcp_nk_method(num_nodes: int, num_colors: int, edges: List[Tuple[int, int]]) -> QuantumCircuit:
    node_ancilla_registers, pairs_register, edge_register, classical_register, circuit = nk_init(num_nodes, num_colors, edges)

    generate_color_matrix(num_colors, circuit, node_ancilla_registers)

    detect_color_conflicts(num_nodes, num_colors, circuit, node_ancilla_registers, pairs_register)

    remove_color_conflicts(num_nodes, num_colors, circuit, node_ancilla_registers, pairs_register, edge_register)

    # Measure the qubits
    for i, (node, _) in enumerate(node_ancilla_registers):
        circuit.measure(node, classical_register[i * num_colors:(i + 1) * num_colors])

    return circuit

nNodes = 4
nColors = 2
qc = gcp_nk_method(nNodes, nColors, [(0, 1)])
qc.draw(output='mpl')

# Simulate the circuit
counts = simulate_circuit(qc)
plot_histogram(counts)

for key, value in counts.items():
    print("Counts: ", value)
    # print key as nNodes x nColors matrix
    for n in range(nNodes):
        for k in range(nColors):
            print(key[::-1][n*nColors+k],end=' ')
        print('')

In [ ]:
def outer_sum(color_array: np.ndarray) -> np.ndarray:
    result = []

    for color_idx in range(color_array.shape[1]):
        color_k_matrix = np.stack([color_array[:, color_idx]] * color_array.shape[0])
        result.append(color_k_matrix + color_k_matrix.T)

    return np.concatenate(result, axis=1)


In [ ]:
C = np.array([
    [1, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
])
G = np.array([
    [0, 0, 1, 1],
    [0, 0, 1, 1],
    [1, 1, 0, 1],
    [1, 1, 1, 0],
])


outer_sum(C) * np.concatenate([G] * C.shape[1], axis=1)

In [ ]:
# N = number of nodes
# K = number of colours
'''
In this method we use a colouring matrix:
1 line for each node
1 column for each colour
one and only one 1 in each line, other values 0
It means we need at least NK qubits to describe such a matrix
'''

nNodes=2
nColors=2 # nColors <= nNodes
nc=nColors+1 # For each node, nColors qubits that will be measured, +1 ancillary
nn2=round((nNodes-1)*nNodes/2) # Number of pairs of different nodes and of possible edges
sc=round(nc*nNodes) # Number of qubits to skip before the ancillary qubits for pairs and edges
sg=round(nc*nNodes + nn2) # Beginning of the list of qubits that describe the graph
nqbits=sc + 2*nn2 # Total number of qubits
print("nNodes=",nNodes,"nColors=",nColors,"nc=",nc,"nn2=",nn2,"sc=",sc,"sg=",sg,"nqbits=",nqbits)
# Create a Quantum Circuit
q = QuantumRegister(nqbits)
c=ClassicalRegister(nColors*nNodes) # To measure to "extract" the colouring matrices
qc = QuantumCircuit(q,c)
# Add the graph (binary list of nNodes*(nNodes-1)/2 elements,
# because the graph is symmetric and there is no i=>i edges.
# Set to |1> the qubits corresponding to an edge.
'''
3 nodes, needs 3 colors
0 1 1
1 0 1
1 1 0
=> 1 1 1
3 nodes, needs 2 colors
0 1 0
1 0 1
0 1 0
=> 1 0 1
2 nodes
2 n
0 1
1 0
=> 1
4 nodes, needs 3 colors
0 0 1 1
0 0 1 1
1 1 0 1
1 1 1 0
=> 0 1 1 1 1 1
'''
qc.x(sg) # Set to |1> iif there is an edge between node 0 and node 1
# qc.x(sg+1) # Set to |1> iif there is an edge between node 0 and node 2
# qc.x(sg+2) # Set to |1> iif there is an edge between node 1 and node 2
# qc.x(sg+3) # Set to |1> iif there is an edge between node 1 and node 3
# qc.x(sg+4) # Set to |1> iif there is an edge between node 2 and node 3
# qc.x(sg+5) # Set to |1> iif there is an edge between node 2 and node 4
# ----------------------------------------Generate a colouring matrix
# Initialisation
# Hadamard gate for qubits that represent the colouring matrix
s=0
for n in range(nNodes):
    for k in range(nColors):
        qc.h(s+k)
    s=s+nc
#-----------------------------
# Constraints
# A 1 and only one 1 in each of the coloring matrix
qc.barrier()
s=0
for n in range(nNodes):
    for k in range(nColors-1):
        for l in range(k+1,nColors):
            # Eliminate 11
            qc.ccx (s+k,s+l,s+nColors)
            qc.cx (s+nColors,s+k)
            qc.reset(s+nColors)
    # Eliminate 0* (no colour assigned to the node n)
    for k in range(nColors):
        qc.x(s+k) # if 0 => 1.
    cb=list(range(s,s+nColors) )
    qc.mcx(cb,s+nColors)
    for k in range(nColors):
        qc.x(s+k) # if 1 => 0
    qc.cx(s+nColors,s+nColors-1)
    qc.reset(s+nColors)
    s=s+nc
# At this point, if we measure,
# we find colouring matrices (nNodes lines, nColors columns),
# one and only one 1 in each line (a node does have a colour, and only one)
print('end of colouring matrices')
qc.barrier()
# Switch the ancillary qubits corresponding to pairs of nodes
# that have the same colour
for k in range(nColors):
    s=nc*nNodes
    for n1 in range(nNodes-1):
        for n2 in range(n1+1,nNodes):
            n11=nc*n1+k # If q[n11]=|1> it means the node n1 has the color k
            n22=nc*n2+k # If q[n22]=|1> it means the node n2 has the color k
            qc.ccx(n11,n22,s) # If same color k, set s to |1>.
            # Notice it can happens at most for one k
            s=s+1
#print([n11,n22])
# At this point, if we measure the (nNodes-1)*nNodes)/2 ancillary qubits,
# we get binary strings, in which 1 means "same color" for n1 and n2
print('end of pairs of nodes')
qc.barrier()
# Compare to the graph.
for n in range(sc,sc+nn2): # For each pair of nodes
    # If same color and there is an edge "destroy" (set to |0*>) the colouring
    for node in range(nNodes):
        qnode=nc*node
        qnc=qnode+nColors
        for k in range(nColors):
            cb=[n,n+nn2,qnode+k]
            qc.mcx (cb,qnc)
            cb=[n,n+nn2,qnc]
            qc.mcx (cb,qnode+k)
            qc.reset(qnc)
print('end of compare to graph')
qc.barrier()
# Measure (only the qubits describing the colouring matrices)
cb=0
for n in range(nNodes):
    s=n*(nColors+1)
    for k in range(nColors):
        qb=s+k
        qc.measure(qb,cb)
        cb=cb+1
print('end of measures')
qc.draw(output='mpl')

In [ ]:
counts = simulate_circuit(qc, 1024)
plot_histogram(counts)

In [ ]:
for key, value in counts.items():
    print("Counts: ", value)
    # print key as nNodes x nColors matrix
    for n in range(nNodes):
        for k in range(nColors):
            print(key[::-1][n*nColors+k],end=' ')
        print('')

## Grover's algorithm for 3-coloring

In [ ]:
class Grover3GC:
    color_bits = 2

    def __init__(self, num_nodes: int, adjancency_matrix: List[List[int]], num_correct_answers: int):
        self.num_nodes = num_nodes
        self.edges = self._get_edges(adjancency_matrix)
        self.num_qubits = num_nodes * self.color_bits
        self.num_edges = int(num_nodes * (num_nodes - 1) / 2)

        self.edge_combinations = list(combinations(range(self.num_nodes), 2))
        self.num_iterations = round(((np.pi/2)/np.arccos(np.sqrt((num_nodes**3-num_correct_answers)/num_nodes**3)) - 1)/2)

        self._build_circuit()

    def solve(self, shots: int = 1024) -> Dict[str, int]:
        #TODO: Filter results that have only 2 colors
        return simulate_circuit(self.quantum_circuit, shots)

    def _get_edges(self, adjancency_matrix: List[List[int]]) -> List[Tuple[int, int]]:
        edges = []
        for i in range(len(adjancency_matrix)):
            for j in range(i + 1, len(adjancency_matrix)):
                if adjancency_matrix[i][j] == 1:
                    edges.append((i, j))
        return edges

    def _disconnected_nodes(self):
        for n, (n1, n2) in enumerate(self.edge_combinations):
            if (n1, n2) in self.edges:
                continue
            self.quantum_circuit.x(self.edges_register[n])

    def _r00(self):
        for node in range(self.num_nodes):
            self.quantum_circuit.ry(2*np.arcsin(np.sqrt(2/3)), self.nodes_register[node*self.color_bits])
            self.quantum_circuit.ch(self.nodes_register[node*self.color_bits], self.nodes_register[node*self.color_bits+1])
            self.quantum_circuit.x(self.nodes_register[node*self.color_bits+1])
        self.quantum_circuit.barrier()

    def _r00_dg(self):
        for node in range(self.num_nodes):
            self.quantum_circuit.x(self.nodes_register[node*self.color_bits+1])
            self.quantum_circuit.ch(self.nodes_register[node*self.color_bits], self.nodes_register[node*self.color_bits+1])
            self.quantum_circuit.ry(-2*np.arcsin(np.sqrt(2/3)), self.nodes_register[node*self.color_bits])
        self.quantum_circuit.barrier()

    def _oracle(self):
        for n, (n1, n2) in enumerate(self.edge_combinations):
            if (n1, n2) in self.edges:
                self.quantum_circuit.ccx(self.nodes_register[n1*self.color_bits], self.nodes_register[n2*self.color_bits+1], self.edges_register[n])
                self.quantum_circuit.ccx(self.nodes_register[n1*self.color_bits+1], self.nodes_register[n2*self.color_bits], self.edges_register[n])

        self.quantum_circuit.mct(self.edges_register, self.phase_register[0])

        for n, (n1, n2) in enumerate(reversed(self.edge_combinations)):
            if (n1, n2) in self.edges:
                self.quantum_circuit.ccx(self.nodes_register[n1*self.color_bits+1], self.nodes_register[n2*self.color_bits], self.edges_register[self.num_edges-1-n])
                self.quantum_circuit.ccx(self.nodes_register[n1*self.color_bits], self.nodes_register[n2*self.color_bits+1], self.edges_register[self.num_edges-1-n])
        self.quantum_circuit.barrier()

    def _diffuser(self):
        self._r00_dg()

        self.quantum_circuit.x(self.nodes_register)
        self.quantum_circuit.mct(self.nodes_register, self.phase_register)
        self.quantum_circuit.x(self.nodes_register)
        self.quantum_circuit.barrier()

        self._r00()

    def _measure(self):
        self.quantum_circuit.measure(self.nodes_register, self.classical_register)

    def _build_circuit(self):
        self.nodes_register = QuantumRegister(self.num_qubits, 'nodes')
        self.edges_register = QuantumRegister(self.num_edges, 'edges')
        self.phase_register = QuantumRegister(1, 'phase')
        self.classical_register = ClassicalRegister(self.num_qubits, 'classical')
        self.quantum_circuit = QuantumCircuit(self.nodes_register, self.edges_register, self.phase_register, self.classical_register)

        self._disconnected_nodes()

        self.quantum_circuit.h(self.phase_register)
        self.quantum_circuit.z(self.phase_register)

        self._r00()

        for i in range(self.num_iterations):
            self._oracle()
            self._diffuser()

        self._measure()

    @staticmethod
    def _simulate_circuit(circuit: QuantumCircuit, shots: int) -> Dict[str, int]:
        backend = Aer.get_backend('qasm_simulator')
        job = execute(circuit, backend=backend, shots=shots)
        result = job.result()
        counts = result.get_counts(circuit)
        return counts



am = [
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 0]
]

solver = Grover3GC(3, am, 3*2*1)
# solver.quantum_circuit.draw(output='mpl')
print(solver.num_iterations)
counts = solver.solve()
print(counts)
plot_histogram(counts)

In [ ]:
num_nodes = 4
color_bits = 2
num_qubits = num_nodes*color_bits
num_pairs = int(num_nodes*(num_nodes - 1)/2)

qr = QuantumRegister(num_qubits, 'color')
an = QuantumRegister(num_pairs + 1, 'ancilla')
cr = ClassicalRegister(num_qubits, 'classical')
circuit = QuantumCircuit(qr, an, cr)

circuit.x(an[0])
circuit.x(an[1])

circuit.h(an[num_pairs])
circuit.z(an[num_pairs])

for node in range(num_nodes):
    circuit.ry(2*np.arcsin(np.sqrt(2/3)), qr[node*color_bits])
    circuit.ch(qr[node*color_bits], qr[node*color_bits+1])
    circuit.x(qr[node*color_bits+1])
circuit.barrier()

it = round(((np.pi/2)/np.arccos(np.sqrt((num_nodes**3-12)/num_nodes**3)) - 1)/2)
print(it)
for i in range(it):
    for n, (n1, n2) in enumerate(combinations(range(num_nodes), 2)):
        if n not in [0, 1]:
            circuit.ccx(qr[n1*color_bits], qr[n2*color_bits+1], an[n])
            circuit.ccx(qr[n1*color_bits+1], qr[n2*color_bits], an[n])

    circuit.mct(an[:num_pairs], an[num_pairs])

    for n, (n1, n2) in enumerate(reversed(list(combinations(range(num_nodes), 2)))):
        if n not in [num_pairs-2, num_pairs-1]:
            circuit.ccx(qr[n1*color_bits+1], qr[n2*color_bits], an[num_pairs-1-n])
            circuit.ccx(qr[n1*color_bits], qr[n2*color_bits+1], an[num_pairs-1-n])
    circuit.barrier()

    for node in range(num_nodes):
        circuit.x(qr[node*color_bits+1])
        circuit.ch(qr[node*color_bits], qr[node*color_bits+1])
        circuit.ry(-2*np.arcsin(np.sqrt(2/3)), qr[node*color_bits])
    circuit.barrier()

    circuit.x(qr)
    circuit.mct(qr, an[num_pairs])
    circuit.x(qr)
    circuit.barrier()

    for node in range(num_nodes):
        circuit.ry(2*np.arcsin(np.sqrt(2/3)), qr[node*color_bits])
        circuit.ch(qr[node*color_bits], qr[node*color_bits+1])
        circuit.x(qr[node*color_bits+1])
    circuit.barrier()

circuit.measure(qr, cr)

# circuit.draw(output='mpl')

In [ ]:
counts = simulate_circuit(circuit, 1024)
plot_histogram(counts)

In [ ]:
# Get 5 most common results
most_common = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:6]

# Exclude solutions with less than 3 colors
# most_common = 

# print most common results
for key, value in most_common:
    print("Counts: ", value)
    # print key as nNodes x nColors matrix
    for n in range(3):
        for k in range(color_bits):
            print(key[::-1][n*color_bits+k],end=' ')
        print('')

In [ ]:
qr = QuantumRegister(3, 'color')
an = QuantumRegister(1, 'ancilla')
cr = ClassicalRegister(3, 'classical')
qc = QuantumCircuit(qr, an, cr)


qc.h(qr)
qc.barrier()

#000
qc.x(qr)
qc.mct(qr, an)
qc.x(qr)
qc.cx(an, qr[0])
qc.cx(an, qr[1])
qc.cx(an, qr[2])
qc.reset(an)
qc.barrier()

#110
qc.x(qr[0])
qc.mct(qr, an)
qc.x(qr[0])
qc.cx(an, qr[0])
qc.reset(an)
qc.barrier()

#101
qc.x(qr[1])
qc.mct(qr, an)
qc.x(qr[1])
qc.cx(an, qr[1])
qc.reset(an)
qc.barrier()

#011
qc.x(qr[2])
qc.mct(qr, an)
qc.x(qr[2])
qc.cx(an, qr[2])
qc.reset(an)
qc.barrier()




qc.measure(qr, cr)

qc.draw(output='mpl')

In [ ]:
plot_histogram(simulate_circuit(qc, 1024))

In [ ]:
qr = QuantumRegister(3, 'color')
an = QuantumRegister(1, 'ancilla')
cr = ClassicalRegister(3, 'classical')
qc = QuantumCircuit(qr, an, cr)


qc.ry(2*np.arcsin(np.sqrt(2/3)), qr[0])
qc.ch(qr[0], qr[1])
qc.ch(qr[1], qr[2])

qc.cx(qr[1], qr[2])
qc.cx(qr[0], qr[1])

qc.x(qr[0])
qc.x(qr)

qc.barrier()


qc.measure(qr, cr)

qc.draw(output='mpl')

In [ ]:
plot_histogram(simulate_circuit(qc, 1024))

In [ ]:
class Grover2GC:
    color_bits = 1

    def __init__(self, num_nodes: int, adjancency_matrix: List[List[int]], num_correct_answers: int):
        self.num_nodes = num_nodes
        self.edges = self._get_edges(adjancency_matrix)
        self.num_qubits = num_nodes * self.color_bits
        self.num_edges = int(num_nodes * (num_nodes - 1) / 2)

        self.edge_combinations = list(combinations(range(self.num_nodes), 2))
        self.num_iterations = round(((np.pi/2)/np.arccos(np.sqrt((num_nodes**2-num_correct_answers)/num_nodes**2)) - 1)/2)

        self._build_circuit()

    def solve(self, shots: int = 1024) -> Dict[str, int]:
        #TODO: Filter results that have only 2 colors
        return self._simulate_circuit(self.quantum_circuit, shots)

    def _get_edges(self, adjancency_matrix: List[List[int]]) -> List[Tuple[int, int]]:
        edges = []
        for i in range(len(adjancency_matrix)):
            for j in range(i + 1, len(adjancency_matrix[0])):
                if adjancency_matrix[i][j] == 1:
                    edges.append((i, j))
        return edges

    def _disconnected_nodes(self):
        for n, (n1, n2) in enumerate(self.edge_combinations):
            if (n1, n2) in self.edges:
                continue
            self.quantum_circuit.x(self.edges_register[n])

    def _r(self):
        self.quantum_circuit.h(self.nodes_register)
        self.quantum_circuit.barrier()

    def _r00_dg(self):
        self.quantum_circuit.h(self.nodes_register)
        self.quantum_circuit.barrier()

    def _oracle(self):
        for n, (n1, n2) in enumerate(self.edge_combinations):
            if (n1, n2) in self.edges:
                self.quantum_circuit.x(self.nodes_register[n1])
                self.quantum_circuit.ccx(self.nodes_register[n1], self.nodes_register[n2], self.edges_register[n])
                self.quantum_circuit.x(self.nodes_register[n1])

                self.quantum_circuit.x(self.nodes_register[n2])
                self.quantum_circuit.ccx(self.nodes_register[n1], self.nodes_register[n2], self.edges_register[n])
                self.quantum_circuit.x(self.nodes_register[n2])

        self.quantum_circuit.mct(self.edges_register, self.phase_register[0])

        for n, (n1, n2) in enumerate(reversed(self.edge_combinations)):
            if (n1, n2) in self.edges:
                self.quantum_circuit.x(self.nodes_register[n2])
                self.quantum_circuit.ccx(self.nodes_register[n1], self.nodes_register[n2], self.edges_register[self.num_edges-1-n])
                self.quantum_circuit.x(self.nodes_register[n2])

                self.quantum_circuit.x(self.nodes_register[n1])
                self.quantum_circuit.ccx(self.nodes_register[n1], self.nodes_register[n2], self.edges_register[self.num_edges-1-n])
                self.quantum_circuit.x(self.nodes_register[n1])
        self.quantum_circuit.barrier()

    def _diffuser(self):
        self._r00_dg()

        self.quantum_circuit.x(self.nodes_register)
        self.quantum_circuit.mct(self.nodes_register, self.phase_register)
        self.quantum_circuit.x(self.nodes_register)

        self._r()

    def _measure(self):
        self.quantum_circuit.measure(self.nodes_register, self.classical_register)

    def _build_circuit(self):
        self.nodes_register = QuantumRegister(self.num_qubits, 'nodes')
        self.edges_register = QuantumRegister(self.num_edges, 'edges')
        self.phase_register = QuantumRegister(1, 'phase')
        self.classical_register = ClassicalRegister(self.num_qubits, 'classical')
        self.quantum_circuit = QuantumCircuit(self.nodes_register, self.edges_register, self.phase_register, self.classical_register)

        self._disconnected_nodes()

        self.quantum_circuit.h(self.phase_register)
        self.quantum_circuit.z(self.phase_register)

        self._r()

        for i in range(max(1, self.num_iterations)):
            self._oracle()
            self._diffuser()

        self._measure()

    @staticmethod
    def _simulate_circuit(circuit: QuantumCircuit, shots: int) -> Dict[str, int]:
        backend = Aer.get_backend('qasm_simulator')
        job = execute(circuit, backend=backend, shots=shots)
        result = job.result()
        counts = result.get_counts(circuit)
        return counts


am = [
    [0, 1, 0],
    [1, 0, 1],
    [0, 1, 0]
]


solver = Grover3GC(3, am, 6)
# solver.quantum_circuit.draw(output='mpl')
print(solver.num_iterations)
counts = solver.solve()
print(counts)
plot_histogram(counts)

In [ ]:
am = [
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1],
    [1, 0, 0, 1, 1],
    [0, 0, 1, 0, 0],
    [0, 1, 1, 0, 0],
]


solver = Grover2GC(5, am, 2)
solver.quantum_circuit.draw(output='mpl')
plot_histogram(solver.solve())

In [ ]:
# Get 5 most common results
most_common = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:4]

# Exclude solutions with less than 3 colors
# most_common = 

# print most common results
for key, value in most_common:
    print("Counts: ", value)
    # print key as nNodes x nColors matrix
    print(np.array(list(key[::-1])).reshape(5, 1))

In [ ]:
class NKColoring:
    """
    Solves the Graph k-Coloring problem.
    """

    def __init__(self, num_nodes: int, num_colors: int, edges: List[Tuple[int, int]]):
        self.num_nodes = num_nodes
        self.num_colors = num_colors
        self.edges = edges

        self._build_circuit()

    @staticmethod
    def _simulate_circuit(circuit: QuantumCircuit, shots: int) -> Dict[str, int]:
        backend = Aer.get_backend('qasm_simulator')
        job = execute(circuit, backend=backend, shots=shots)
        result = job.result()
        counts = result.get_counts(circuit)
        return counts

    def solve(self, shots=1024):
        return self._simulate_circuit(self.quantum_circuit, shots)

    def _define_registers(self):
        # Create a Quantum Register for each node with num_colors qubits and an ancilla qubit
        self.node_ancilla_registers: List[Tuple[QuantumRegister, QuantumRegister]] = []
        for node in range(self.num_nodes):
            tuple_register = (QuantumRegister(self.num_colors, f"node_{node}"), QuantumRegister(1, f"ancilla_{node}"))
            self.node_ancilla_registers.append(tuple_register)

        self.node_pairs = list(combinations(range(self.num_nodes), 2))

        # Create a Quantum Register for each pair of nodes with 1 qubit
        self.pairs_register: List = []
        for node1, node2 in self.node_pairs:
            self.pairs_register.append(QuantumRegister(1, f"pair_{node1}_{node2}"))

        # Create a Quantum Register for each edge of the graph with 1 qubit
        self.edge_register: List = []
        for node1, node2 in self.node_pairs:
            self.edge_register.append(QuantumRegister(1, f"edge_{node1}_{node2}"))

        # Create a Classical Register with num_nodes bits.
        self.classical_register = ClassicalRegister(self.num_nodes * self.num_colors, "classical")

    def _init_edges(self):
        for edge in self.edges:
            if edge in self.node_pairs:
                self.quantum_circuit.x(self.edge_register[self.node_pairs.index(edge)])
            elif (edge[1], edge[0]) in self.node_pairs:
                self.quantum_circuit.x(self.edge_register[self.node_pairs.index((edge[1], edge[0]))])
            else:
                raise Exception(f"Edge '{edge}' is not a valid edge.")

    def _generate_color_matrix(self):
        # Start with all possible color arrangements
        for node, ancilla in self.node_ancilla_registers:
            self.quantum_circuit.h(node)

        for node_colors, ancilla in self.node_ancilla_registers:

            # Remove all arrangements where one node has more than one color
            for n1, n2 in combinations(range(self.num_colors), 2):
                color1 = node_colors[n1]
                color2 = node_colors[n2]

                self.quantum_circuit.ccx(color1, color2, ancilla)
                self.quantum_circuit.cx(ancilla, color1)
                self.quantum_circuit.reset(ancilla)

            # Remove arrangement where node has no color
            self.quantum_circuit.x(node_colors)
            self.quantum_circuit.mct(node_colors, ancilla)
            self.quantum_circuit.x(node_colors)
            self.quantum_circuit.cx(ancilla, node_colors[-1])
            self.quantum_circuit.reset(ancilla)
        self.quantum_circuit.barrier()

    def _detect_color_conflicts(self):
        for n, (n1, n2) in enumerate(self.node_pairs):
            for i in range(self.num_colors):
                node1_color = self.node_ancilla_registers[n1][0][i]
                node2_color = self.node_ancilla_registers[n2][0][i]

                self.quantum_circuit.ccx(node1_color, node2_color, self.pairs_register[n])
        self.quantum_circuit.barrier()

    def _resolve_color_conflicts(self):
        for n, (edge, pair) in enumerate(zip(self.edge_register, self.pairs_register)):
            for node, ancilla in self.node_ancilla_registers:
                for i in range(self.num_colors):
                    node_color = node[i]

                    self.quantum_circuit.mct([edge, pair, node_color], ancilla)
                    self.quantum_circuit.cx(ancilla, node_color)
                    self.quantum_circuit.reset(ancilla)
        self.quantum_circuit.barrier()

    def _measure(self):
        for i, (node, _) in enumerate(self.node_ancilla_registers):
            self.quantum_circuit.measure(node, self.classical_register[i * self.num_colors:(i + 1) * self.num_colors])

    def _build_circuit(self):
        self._define_registers()

        self.quantum_circuit = QuantumCircuit(
            *[reg for reg_tuple in self.node_ancilla_registers for reg in reg_tuple], 
            *self.pairs_register, 
            *self.edge_register,
            self.classical_register
        )

        self._init_edges()

        self._generate_color_matrix()

        self._detect_color_conflicts()

        self._resolve_color_conflicts()

        self._measure()

nNodes = 3
nColors = 3
nk_solver = NKColoring(nNodes, nColors, [(0,1)])
nk_solver.quantum_circuit.draw(output='mpl')

# Simulate the circuit
counts = nk_solver.solve()
plot_histogram(counts)

for key, value in counts.items():
    print("Counts: ", value)
    # print key as nNodes x nColors matrix
    for n in range(nNodes):
        for k in range(nColors):
            print(key[::-1][n*nColors+k],end=' ')
        print('')

In [ ]:
am = [
    [0, 1, 1, 0, 1, 1],
    [1, 0, 1, 1, 0, 0],
    [1, 1, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 1],
    [1, 0, 1, 0, 0, 0],
    [1, 0, 0, 1, 0, 0]
]

solver = Grover3GC(6, am, 18)

In [ ]:
counts = solver.solve()
plot_histogram(counts)

In [ ]:
# Get 5 most common results
most_common = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:18]

# Exclude solutions with less than 3 colors
# most_common = 

# print most common results
for key, value in most_common:
    print("Counts: ", value)
    # print key as nNodes x nColors matrix
    for n in range(6):
        for k in range(2):
            print(key[::-1][n*2+k],end=' ')
        print('')